In [1]:
import numpy as np
import pandas as pd
import glob
import os

In [2]:
id_to_type_path = r"/mnt/c/Users/donna/Downloads/Thesis/brca_correlaties/id_to_type.xlsx"
TCGA_matrix_path = r"/mnt/c/Users/donna/Downloads/Thesis/brca_correlaties/TCGA_matrix.xlsx"
peak_info_path = r"/mnt/c/Users/donna/Downloads/Thesis/brca_correlaties/TCGA_peakcalls.xlsx"

In [3]:
def get_first_three_parts(sample_id):
    return '-'.join(sample_id.split('-')[:3])

In [5]:
tcga_data = pd.read_csv(r"/mnt/c/Users/donna/Downloads/Thesis/brca_correlaties/TCGA_matrix.csv")

In [6]:
sample_ids = tcga_data.columns[1:].values
full_peak_names = tcga_data["sample"].values
peak_values = tcga_data.iloc[:,1:].values

In [7]:
peak_names = pd.read_excel(peak_info_path, sheet_name="Pan-Cancer_PeakCalls", skiprows=22, usecols = ["Chromosome", "Peak_Name"]) 
filtered_peak_names = peak_names[~peak_names["Chromosome"].isin(["chrX", "chrY"])]["Peak_Name"]
filtered_set = set(filtered_peak_names)
peak_mask = np.array([peak in filtered_set for peak in full_peak_names])

In [8]:
# Save peak names, same for all samples/cancer types
filtered_peaks = full_peak_names[peak_mask]
np.savetxt('/mnt/c/Users/donna/Downloads/Thesis/brca_correlaties/TCGA_data/filtered_peak_names.csv', filtered_peaks, delimiter=",", fmt="%s")

In [9]:
# MAKING MATRIX FOR ONLY THE 10000 PEAKS WITH HIGHEST VARIANCE (first atac, then ff (griffin mean))

In [25]:
# Selecting 10000 high var atac peaks
high_var_peaks = pd.read_csv(r'/mnt/c/Users/donna/Downloads/Thesis/brca_correlaties/TCGA_data/high_var_peaks.csv', header=None).values

In [28]:
print(len(high_var_peaks))

10000


In [35]:
high_var_mask = np.isin(full_peak_names, high_var_peaks)

In [48]:
# Selecting 1000 high var ff (covx) peaks
ff_high_var_peaks = pd.read_csv(r'/mnt/c/Users/donna/Downloads/Thesis/brca_correlaties/TCGA_data/high_var_covx_peaks.csv', header=None).values

In [49]:
ff_high_var_mask = np.isin(full_peak_names, ff_high_var_peaks)

In [9]:
## FETCHING CANCER SPECIFIC SAMPLES
# -> Change cancer type!!!

In [54]:
id_to_type = pd.read_excel(id_to_type_path, header=0, sheet_name='Sheet1', dtype='object')
selected_ids = id_to_type[id_to_type.iloc[:, 1].isin(['COAD'])]['Sample_id']

In [55]:
def get_first_three_parts(sample_id):
    sample_id = sample_id.astype(str)
    split_parts = np.char.split(sample_id, '-')
    return np.array(['-'.join(parts[:3]) for parts in split_parts])

In [56]:
first_three_id = get_first_three_parts(sample_ids)
presence_array = np.isin(first_three_id, selected_ids)

In [39]:
# -> Change output name to correct cancer type

In [46]:
selected_peak_values = peak_values[np.ix_(peak_mask, presence_array)]
np.savetxt('/mnt/c/Users/donna/Downloads/Thesis/brca_correlaties/TCGA_data/brca_atac_data.csv', selected_peak_values, delimiter=",", fmt="%.8f") 

In [34]:
# -> Change output name to correct cancer type

In [17]:
selected_sample_ids = sample_ids[presence_array]
np.savetxt('/mnt/c/Users/donna/Downloads/Thesis/brca_correlaties/TCGA_data/brca_sample_ids.csv', selected_sample_ids, delimiter=",", fmt="%s") 

In [47]:
# Only 10000 high var peaks
# -> Change output name to correct cancer type
high_var_peak_values = peak_values[np.ix_(high_var_mask, presence_array)]
np.savetxt('/mnt/c/Users/donna/Downloads/Thesis/brca_correlaties/TCGA_data/high_var_brca.csv', high_var_peak_values, delimiter=",", fmt="%.8f") 

In [57]:
# Only 10000 high covxvar peaks
# -> Change output name to correct cancer type
ff_high_var_peak_values = peak_values[np.ix_(ff_high_var_mask, presence_array)]
np.savetxt('/mnt/c/Users/donna/Downloads/Thesis/brca_correlaties/TCGA_data/high_covxvar_coad.csv', ff_high_var_peak_values, delimiter=",", fmt="%.8f") 